## 이 문서를 수정할 당신에게...
#### 현재 상황은 아래와 같습니다.
1. 우리의 앱에서 중요한 요소 중 하나는 와인을 추천하고, 이를 실제로 구매할 수 있는 링크에 연결하는 것입니다. 또, 실제 와인바를 추천해주기도 합니다. 이를 위해서 실제 와인을 파는 사이트와 와인바를 데이터베이스로 만들어두어야하기 때문에 이를 위해서는 크롤링이 필요합니다.

#### 당신의 목표는 아래와 같습니다.
1. 아래 사이트에 대한 크롤링을 진행합니다. 크롤링이 필요한 요소는 와인바 추천에 필요하다고 생각하는 모든 것을 하면 됩니다. 또한 추가로 더 좋은 사이트의 크롤링도 좋습니다.

    <strong>와인바</strong>
    - 서울내에 있는 와인바(사이트 찾아야함)

2. 와인바에 대한 데이터베이스도 통합합니다. 추가로 다른 schema도 추가해줍니다.
- 상호명
- 위치(위도, 경도)
- 도로명주소
- 메뉴
- 영업시간

#### 참고 사항
현재 작성된 크롤링은 아래의 프레임워크로 작업했습니다.
- [Scrapy](https://docs.scrapy.org/en/latest/intro/tutorial.html)
- [Selenium](https://selenium-python.readthedocs.io/)

In [1]:
import os
from time import sleep
import configparser

from tqdm import tqdm
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

아래 코드를 실행하면 크롬이 열리고, 와인앤모어 로그인 페이지로 접속하게 된다.

In [16]:
# Setup the driver. This one uses chrome with some options and a path to the chromedriver
# driver = webdriver.Chrome(ChromeDriverManager().install())
driver = webdriver.Chrome()

# implicitly_wait tells the driver to wait before throwing an exception
driver.implicitly_wait(30)

# go to the login page
driver.get('https://www.mangoplate.com/search/%EC%99%80%EC%9D%B8?keyword=%EC%99%80%EC%9D%B8&page=1') # 망고플레이트 와인 검색 창으로 이동

In [8]:
driver.get('https://www.mangoplate.com/search/%EC%99%80%EC%9D%B8?keyword=%EC%99%80%EC%9D%B8&page=1')

In [61]:
def get_detail_info(detail_url):
    driver.get(detail_url)
    # Allow time for the website to load
    sleep(10)
    data_dict = {}
    data_dict['url'] = driver.current_url

    name = driver.find_element(By.CSS_SELECTOR, "h1.restaurant_name")
    data_dict['name'] = name.text if name else ''

    rating = driver.find_element(By.CSS_SELECTOR, "strong.rate-point span")
    data_dict['rating'] = rating.text if rating else ''

    try:
        address = driver.find_element(By.XPATH, '//tr[th[contains(text(), "주소")]]/td')
        data_dict['address'] = address.text if address else ''
    except:
        data_dict['address'] = ''

    try:
        phone = driver.find_element(By.XPATH, '//tr[th[contains(text(), "전화번호")]]/td')
        data_dict['phone'] = phone.text if phone else ''
    except:
        data_dict['phone'] = ''

    # 가격대
    try:
        price_range = driver.find_element(By.XPATH, '//tr[th[contains(text(), "가격대")]]/td')
        data_dict['price_range'] = price_range.text if price_range else ''
    except:
        data_dict['price_range'] = ''


    # 주차
    try:
        parking = driver.find_element(By.XPATH, '//tr[th[contains(text(), "주차")]]/td')
        data_dict['parking'] = parking.text if parking else '' 
    except:
        data_dict['parking'] = ''

    # 영업시간
    try:
        opening_hours = driver.find_element(By.XPATH, '//tr[th[contains(text(), "영업시간")]]/td')
        data_dict['opening_hours'] = opening_hours.text if opening_hours else ''
    except:
        data_dict['opening_hours'] = ''

    # 휴일
    try:
        holidays = driver.find_element(By.XPATH, '//tr[th[contains(text(), "휴일")]]/td')
        data_dict['holidays'] = holidays.text if holidays else ''
    except:
        data_dict['holidays'] = ''

    # 메뉴
    menu_items = driver.find_elements(By.XPATH, '//tr[th[contains(text(), "메뉴")]]//li')
    menu = {}
    for item in menu_items:
        menu_name = item.find_element(By.CLASS_NAME, 'Restaurant_Menu').text
        menu_price = item.find_element(By.CLASS_NAME, 'Restaurant_MenuPrice').text
        menu[menu_name] = menu_price
    data_dict['menu'] = menu

    img_elements = driver.find_elements(By.XPATH, '//img[@class="center-croping"]')

    # 각 이미지 요소에서 'src' 속성을 추출합니다.
    img_urls = [img.get_attribute('src') for img in img_elements]
    data_dict['img_urls'] = img_urls

    reviews = driver.find_elements(By.CSS_SELECTOR, 'p.RestaurantReviewItem__ReviewText')
    review = {}
    review_list = []
    for review in reviews:
        review_list.append(review.text)
    data_dict['review'] = review_list   

    return data_dict

In [54]:
get_detail_info('https://www.mangoplate.com/restaurants/Ha-RjMRFLWAZ')

{'url': 'https://www.mangoplate.com/restaurants/Ha-RjMRFLWAZ',
 'name': '미도림',
 'rating': '3.9',
 'address': '서울특별시 성동구 왕십리로 108 3F\n지번 서울시 성동구 성수동1가 656-1069 3F',
 'phone': '02-469-8486',
 'price_range': '만원-2만원',
 'parking': '주차공간없음',
 'opening_hours': '월-금: 16:00 - 24:00\n토-일: 13:00 - 01:00',
 'holidays': '',
 'menu': {'취나물파스타': '14,000원',
  '부추교자': '12,000원',
  '더덕약고추장 곁들인 새우전변': '12,000원',
  '미더덕소스로 버무린 육회': '17,000원',
  '성게파테와 기정떡': '13,000원'},
 'img_urls': ['https://mp-seoul-image-production-s3.mangoplate.com/572183_1624961579224137.jpg?fit=around|512:512&crop=512:512;*,*&output-format=jpg&output-quality=80',
  'https://mp-seoul-image-production-s3.mangoplate.com/576647_1675389463747207.jpg?fit=around|512:512&crop=512:512;*,*&output-format=jpg&output-quality=80',
  'https://mp-seoul-image-production-s3.mangoplate.com/576647_1675389464599858.jpg?fit=around|512:512&crop=512:512;*,*&output-format=jpg&output-quality=80',
  'https://mp-seoul-image-production-s3.mangoplate.com/576647_

아래 코드를 실행하면 페이지를 순회하면 크롤링을 진행하고, 이를 data_list에 저장한다.

In [42]:
# Navigate to product list page
data_list = []
links = set()
for idx in range(1, 11):
    driver.get(f'https://www.mangoplate.com/search/%EC%99%80%EC%9D%B8?keyword=%EC%99%80%EC%9D%B8&page={idx}')
    sleep(10)
    product_links = driver.find_elements(By.CSS_SELECTOR, "a.only-desktop_not")
    for link in product_links:
        links.add(link.get_attribute('href'))
    for i in product_links:
        if i.get_attribute('href') not in links:
            links.add(i.get_attribute('href'))

In [63]:
links_list = list(links)

In [66]:
cnt = 0
for detail_url in tqdm(links):
    cnt += 1
    if cnt <= 3:
        continue
    data_list.append(get_detail_info(detail_url))

100%|██████████| 200/200 [3:13:29<00:00, 58.05s/it]   


In [69]:
import json

In [70]:
with open('./data/wine_bar.json', 'w', encoding='utf-8') as json_file:
    for data in data_list:
        json_file.write(json.dumps(data, ensure_ascii=False) + '\n')

In [71]:
driver.close()

### wrtn을 이용한 리뷰 데이터 요약

In [4]:
import pandas as pd

In [5]:
df = pd.read_json('./data/wine_bar.json', encoding='utf-8', lines=True)

In [6]:
for idx in len(df['review']):
    df['review'][idx] = df['review'][idx].replace('\n', '')

TypeError: 'int' object is not iterable

In [ ]:
import time

In [2]:
# Setup the driver. This one uses chrome with some options and a path to the chromedriver
# driver = webdriver.Chrome(ChromeDriverManager().install())
driver = webdriver.Chrome()

# implicitly_wait tells the driver to wait before throwing an exception
driver.implicitly_wait(30)

# # go to the login page
# driver.get('https://wrtn.ai/') 

In [9]:
df['summary'] = ''
for idx in range(len(df['review'])):
    driver.get('https://wrtn.ai/') 
    sleep(10)
    input_textarea = driver.find_element(By.CSS_SELECTOR, "div.sc-gsnTZi > textarea")

    llm_model_buttons = driver.find_elements(By.CSS_SELECTOR, "div.sc-hFvvYD")

    for llm_model_button in llm_model_buttons:
        if llm_model_button.text == 'GPT-4':
            break  # 원하는 요소를 찾았으므로 반복문을 종료합니다.

    submit_buttons = driver.find_element(By.CSS_SELECTOR, "div.sc-gsnTZi.sc-lkcIho")
    llm_model_button.click()

    prompt = f"""이 리뷰를 보고, 가게 정보를 요약해줘: {df['review'][idx]}"""
    input_textarea.send_keys(prompt)
    
    submit_buttons.click()
    sleep(40)
    review_elements = driver.find_elements(By.CSS_SELECTOR, "div.sc-gauFBm.ecjxKC>p")
    review_text = ''
    for i in range(len(review_elements)):
        review_text += review_elements[i].text
    review_text= review_text.replace(prompt, '')
    print(review_text)
    df['summary'][idx] = review_text


요약: 미도림은 성수동에 위치한 와인바로, 와인과 다양한 안주가 제공됩니다. 간판이 눈에 잘 띄지 않아 찾기 어려울 수 있으나, 스태프들은 친절하며 음식은 맛있고 고급스럽습니다. 양은 많지 않지만 다양한 안주를 즐길 수 있어서 좋습니다. 주차는 불가능하며 노키드존이란 점을 참고해야 합니다. 음식과 서비스에 대한 리뷰는 각기 다양하였지만, 재방문 의사가 없는 사람도 있습니다.


C:\Users\chois\AppData\Local\Temp\ipykernel_20100\3251060652.py:27: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['summary'][idx] = review_text


해당 가게는 사브서울이라는 이름으로, 고객들은 맛집이면서 예약이 어렵지 않다고 언급하고 있습니다. 와인 선택이 다양하고 가격이 저렴한 것이 눈에 띄며, 와인나라 멤버쉽을 가입하면 추가 할인을 받을 수 있습니다. 가게 인테리어는 깔끔하고 예쁘며, 음식은 다양하고 복합적인 맛이 있는 것으로 평가됩니다. 한우 타르타르, 홍새우 리조또, 꽈리고추, 양갈비스테이크 및 방울양배추 등의 메뉴를 주문하며, 전반적으로 만족한 고객들의 말로 인해 인기 있는 음식점으로 보입니다. 그러나 예약이 어려운 점이 단점으로 지적되며, 대체로 양이 적은 편이라는 평도 있습니다.
이 가게는 칵테일 바로, 고객들은 칵테일과 음식 모두 만족하며, 바텐더는 술에 대한 지식이 풍부하고 친절한 것으로 평가받아 좋은 인상을 남깁니다. 캐주얼한 분위기와 퀄리티 있는 칵테일을 중점으로 소개되며, 혼자서 가도 즐기기 좋다고 합니다. 메뉴판이 없지만, 클래식 칵테일이나 위스키 등을 주문할 수 있으며, 맛과 완성도가 높게 평가됩니다. 또한, 초보자들도 바텐더에게 요청하면 어울리는 칵테일을 추천해 주신다고 합니다. 전반적으로 대학가 상권에 위치하여 부담없이 가기 좋은 친절한 분위기의 칵테일 바라고 평가됩니다.
이 가게는 블라인드 와인 테이스팅을 주요한 경험으로 제공합니다. 다섯 가지 와인을 한 번에 시음할 수 있으며, 화이트와 레드 와인 선택이 가능합니다. 포도 품종과 발효 과정 등의 상세한 정보를 들을 수 있어, 와인을 많이 모르는 사람들도 즐길 수 있습니다. 안주는 간단한 치즈, 하몽, 크래커 등이 제공됩니다. 직원들의 세심한 설명과 친절함이 돋보이며, 와인 맛은 전반적으로 만족스러운 평가를 받았습니다. 다만 일부 손님들은 와인 중 일부가 자신의 기호에 맞지 않았다고 했습니다. 가격은 4.5~5만원 정도이며, 신선한 경험을 찾는 고객들에게 적합한 곳입니다.
이 가게는 수도원 맥주와 와인 전문점으로, 독특한 분위기를 가지고 있습니다. 높은 천장과 어두운 분위기 때문에 고요한 대화를 할 수 있는 환경이 마련되어 

WebDriverException: Message: disconnected: not connected to DevTools
  (failed to check if window was closed: disconnected: not connected to DevTools)
  (Session info: chrome=114.0.5735.199)
Stacktrace:
Backtrace:
	GetHandleVerifier [0x0025A813+48355]
	(No symbol) [0x001EC4B1]
	(No symbol) [0x000F5358]
	(No symbol) [0x000E7D96]
	(No symbol) [0x000E7AB9]
	(No symbol) [0x000F67C0]
	(No symbol) [0x0014C4D8]
	(No symbol) [0x0013A536]
	(No symbol) [0x001182DC]
	(No symbol) [0x001193DD]
	GetHandleVerifier [0x004BAABD+2539405]
	GetHandleVerifier [0x004FA78F+2800735]
	GetHandleVerifier [0x004F456C+2775612]
	GetHandleVerifier [0x002E51E0+616112]
	(No symbol) [0x001F5F8C]
	(No symbol) [0x001F2328]
	(No symbol) [0x001F240B]
	(No symbol) [0x001E4FF7]
	BaseThreadInitThunk [0x76A37D59+25]
	RtlInitializeExceptionChain [0x774CB74B+107]
	RtlClearBits [0x774CB6CF+191]
	(No symbol) [0x00000000]


In [69]:
driver.find_elements(By.CSS_SELECTOR, "div.sc-gauFBm.ecjxKC>p")[2].text

'노키드존으로 아이들과 함께 방문하는 것은 부적합하다는 경험담이 있으며, 안주의 양은 적지만 다양한 메뉴를 시험해볼 수 있어 좋은 평가를 받습니다. 요리들은 대체로 훌륭하다고 평가 받지만 서비스와 분위기에 관해서는 의견이 갈리는 부분들이 있습니다. 일부 고객들은 이용 시간 안내에 대해 불만족을 표현하며, 재방문의사가 없다고 언급하고 있습니다.'

In [63]:
df['summary'][1]

"이 리뷰를 보고, 가게 정보를 요약해줘: ['맛집을 좋아하긴 하지만 그렇다고 예약이 치열하거나 웨이팅이 심한 곳은 별로 탐하지 않는 편.\\n사브서울이 내게도 아직은 그런 곳 중 하나였는데, 우연히 캐치테이블 들어갔더니 생각했던 날짜에 자리가 있었고 냉큼 예약해서 토요일 저녁에 다녀왔다.\\n사브서울을 최대한으로 즐기고 싶어서 1층 와인나라에서 와인 세병을 샀다. 바틀 한병을 시키면 콜키지 3병까지 무료로 이용할 수 있다.\\n화와 1, 레와 2를 사고 사브서울에선 샴페인을 주문해서 샴팡으로 스타트, 음식에 맞춰 레와를 같이 먹는다고 말슴드려 스스로 와인 페어링을 했다.ㅋㅋㅋ\\n한우타르타르,꽈리고추,홍고추리조또,양갈비스테이크, 방울양배추,를 시켰다.\\n라비올로도 먹어보려 했는데 나중엔 솔드아웃이었다. 인기메뉴는 먼저 솔드아웃되기도 하는 것 같으니 처음에 많이 시키는게 좋은거같다.\\n베스트픽은 한우타르타르. 큰송이버섯, 한우, 소스가 어우러져 향과 풍미가 너무 좋았다. 가평잣도 들어가 고소하면서도 부드러운 식감 사이 나름의 크런치함이 심심치 않았다. 홍새우 리조또는 위에 올라간 빨간 소스가 약간 curry스러운 이국적 느낌을 자아냈는데 고트치즈도 응근하게 들어가 약간 눅진한 맛과 알알한 리조또 식감이 재밌고 맛있었다. 홍고추도 통통해서 여러식감이 밸런스가 좋았다.\\n꽈리고추의 훈연한 향은 처음에 매우 임팩트가 컸고 우도땅콩소스가 고소함까지 더해주어 가볍고 또 맛있는 메뉴였다. 맛은 있었지만 재방문시 또 시킬 것 같진 않다. 하지만 시키는 건 추천\\n방울 양배추는, 그 동글동글한 양배추 구이가 나오는 따뜻한 메뉴인 줄 알고 시켰는데, 블루치즈와 나주배등이 곁들여져 채썰어 나오는 콜드메뉴였다. 블루치즈의 꼬릿함, 나주배의 향긋함과 달콤함, 양배추의 상큼함이 어우러져 입가심하기에도 좋았는데 생각지 못한 방향이어서 다소 당황스러웠다. ㅎㅎ이름에서 좀더 유추가능하게 뭔가 바꿔주면 좋겠다. 양갈비 구이는 엄청엄청 부드러웠다. 이제까지 먹어보지 못한 양갈비 느낌이